## 1. Importar Bibliotecas

In [ ]:
import pandas as pd
import json
from collections import Counter
import requests
import csv

## 2. Coleta de Dados da API

In [ ]:
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjce/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="

payload = json.dumps({
    "size": 10000,
    "query": {"match": {"assuntos.codigo": "12487"}},
    "sort": [{"dataAjuizamento": {"order": "desc"}}]
})

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dados_dict = response.json()
print(f"Total de processos encontrados: {dados_dict['hits']['total']['value']}")

# Salvando em Json
with open("dados_completos.json", "w", encoding="utf-8") as f:
    json.dump(dados_dict, f, indent=2, ensure_ascii=False)
print("Dados completos salvos em 'dados_completos.json'")

# Salvando em csv
processos = dados_dict["hits"]["hits"]

numeros = []
for processo in processos:
    numero_processo = processo["_source"]["numeroProcesso"]
    numeros.append(numero_processo)

Total de processos encontrados: 6046


FileNotFoundError: [Errno 2] No such file or directory: '/input/dados_completos.json'

## 3. Processamento dos Dados (Python Dataframe)

In [35]:
# Extrair dados dos processos
processos = []

for hit in dados_dict['hits']['hits']:
    processo = hit['_source']
    numero_processo = processo['numeroProcesso']
    grau = processo['grau']
    classe = processo['classe']['nome']
    assuntos = processo['assuntos']
    data_ajuizamento = processo['dataAjuizamento']
    ultima_atualizacao = processo['dataHoraUltimaAtualizacao']
    formato = processo['formato']['nome']
    codigo = processo['orgaoJulgador']['codigo']
    orgao_julgador = processo['orgaoJulgador']['nome']
    municipio = processo['orgaoJulgador']['codigoMunicipioIBGE']
    
    try:
        movimentos = processo['movimentos']
    except:
        movimentos = []
    
    processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, formato,
                     codigo, orgao_julgador, municipio, grau, assuntos, movimentos])

df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao',
                                      'formato', 'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos'])

print(f"Total de processos no DataFrame: {len(df)}")
df.sample(5)

Total de processos no DataFrame: 6046


,numero_processo,classe,data_ajuizamento,ultima_atualizacao,formato,codigo,orgao_julgador,municipio,grau,assuntos,movimentos
2479,02490315720238060001,Procedimento Comum Cível,20230725000000,2024-10-09T10:06:58.529Z,Eletrônico,8497,21ª VARA CIVEL DA COMARCA DE FORTALEZA,2304400,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 18, 'va..."
4852,06220185520228060000,Agravo de Instrumento,20220210000000,2024-08-20T15:01:34.500Z,Eletrônico,89166,GADES - FRANCISCO LUCIDIO DE QUEIROZ JUNIOR,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 2, 'val..."
5074,06363566820218060000,Agravo de Instrumento,20211104000000,2024-08-20T14:17:52.105Z,Eletrônico,84275,GADES - CARLOS AUGUSTO GOMES CORREIA,2304400,G2,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 4, 'val..."
108,30011924420258060115,Procedimento Comum Cível,20250711000000,2025-10-13T12:27:43.218Z,Eletrônico,82391,1ª VARA CIVEL DA COMARCA DE LIMOEIRO DO NORTE,2307601,G1,"[{'codigo': 12487, 'nome': 'Fornecimento de me...","[{'complementosTabelados': [{'codigo': 19, 'va..."
2003,02773096820238060001,Cumprimento de sentença,20231117000000,2025-09-10T20:33:09.973Z,Eletrônico,90332,NUCLEO DE JUSTIÇA 4.0 - CUMPRIMENTO DE SENTENÇ...,0,G1,"[{'codigo': 12416, 'nome': 'Tutela de Urgência...","[{'complementosTabelados': [{'codigo': 4, 'val..."


## 4. Análise de Decisões
#### Aqui podemos restringir os termos para "procedente/improcedente" ou expandir

In [36]:
# Extrair decisões dos movimentos
decisoes_por_processo = []
tipos_decisao_contagem = []

for idx, row in df.iterrows():
    numero = row['numero_processo']
    movimentos = row['movimentos']
    
    decisoes_encontradas = []
    
    if movimentos:
        for mov in movimentos:
            nome_mov = mov.get('nome', '')
            
            termos_decisao = [
                "Procedência",
                "Improcedência",
                "Improcedência do pedido e improcedência do pedido contraposto"
            ]
            
            # Identificar decisões
            if any(palavra in nome_mov for palavra in termos_decisao):
                decisoes_encontradas.append(nome_mov)
                tipos_decisao_contagem.append(nome_mov)
    
    if decisoes_encontradas:
        decisoes_por_processo.append({
            'numero_processo': numero,
            'decisoes': decisoes_encontradas
        })

print(f"Processos com decisões: {len(decisoes_por_processo)} de {len(df)}")
print(f"\nTipos de decisões encontradas:")
for tipo, count in Counter(tipos_decisao_contagem).most_common(10):
    print(f"  {tipo}: {count}")

Processos com decisões: 2212 de 6046

Tipos de decisões encontradas:
  Procedência: 1616
  Procedência em Parte: 305
  Improcedência: 259
  Procedência em parte do pedido e improcedência do pedido contraposto: 30
  Improcedência do pedido e improcedência do pedido contraposto: 14
  Procedência do pedido e improcedência do pedido contraposto: 5
  Procedência do Pedido - Reconhecimento pelo réu: 3
  Procedência em parte do pedido e procedência do pedido contraposto: 1
  Procedência do pedido e procedência do pedido contraposto: 1


In [37]:
# Criar DataFrame de decisões
decisoes_lista = []

for item in decisoes_por_processo:
    for decisao in item['decisoes']:
        decisoes_lista.append({
            'numero_processo': item['numero_processo'],
            'tipo_decisao': decisao
        })

df_decisoes = pd.DataFrame(decisoes_lista)
df_decisoes.head(10)

,numero_processo,tipo_decisao
0,30039390220258060071,Improcedência
1,02187331420258060001,Procedência
2,30465508320258060001,Procedência em Parte
3,30415587920258060001,Procedência em Parte
4,30048329320258060167,Procedência
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
8,30006925120258060220,Improcedência
9,30303751420258060001,Procedência em Parte


## 5. Seleção Estratificada de Registros
#### Pegar todos os "Improcedência" (259 + 14) e 273 "Procedência" de forma distribuída

In [44]:
# Separar decisões por tipo
df_procedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Procedência'].copy()
df_improcedencia = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência'].copy()
df_improcedencia_contraposto = df_decisoes[df_decisoes['tipo_decisao'] == 'Improcedência do pedido e improcedência do pedido contraposto'].copy()

# Registros de Improcedência (273 registros: 259 + 14)
df_improcedencias_todos = pd.concat([df_improcedencia, df_improcedencia_contraposto])

# Seleção estratificada de Procedência (273 registros)
procedencias = []
tamanho_bloco = 20
pulo = 60
posicao = 0
total_procedencia = len(df_procedencia)

while len(procedencias) < 361 and posicao < total_procedencia:
    # Pegar um bloco de 20 registros
    fim_bloco = posicao + tamanho_bloco
    # Colocar índices de i variando de posicao até fim_bloco na lista "procedencias"
    procedencias.extend(range(posicao, fim_bloco))

    # Pular 100 registros
    posicao = fim_bloco + pulo
    
# Garantir que temos exatamente 273 ao invés de 280
procedencias = procedencias[:361]

# Selecionar os registros de Procedência
df_procedencia_selecionada = df_procedencia.iloc[procedencias].copy()

# Combinar todos os registros selecionados
df_decisoes_balanceado = pd.concat([df_procedencia_selecionada, df_improcedencias_todos])

print(f"\nTotal de registros no dataset balanceado: {len(df_decisoes_balanceado)}")
print(f"  - Procedência: {len(df_procedencia_selecionada)}")
print(f"  - Improcedência: {len(df_improcedencia)}")
print(f"  - Improcedência do pedido e improcedência do pedido contraposto: {len(df_improcedencia_contraposto)}")

# Mostrar distribuição
df_decisoes_balanceado.head(10)


Total de registros no dataset balanceado: 634
  - Procedência: 361
  - Improcedência: 259
  - Improcedência do pedido e improcedência do pedido contraposto: 14


,numero_processo,tipo_decisao
1,02187331420258060001,Procedência
4,30048329320258060167,Procedência
5,02162085920258060001,Procedência
6,30355412720258060001,Procedência
7,30022788520258060071,Procedência
10,30292622520258060001,Procedência
14,30012724320258060071,Procedência
15,30193154420258060001,Procedência
16,30186884020258060001,Procedência
17,30182752720258060001,Procedência


## 6. Exportar Resultados

In [45]:
# Salvar DataFrames em CSV
df.to_csv('processos_completo.csv', sep=',', header=True, index=False)
df_decisoes_balanceado.to_csv('decisoes_resumo.csv', sep=',', header=True, index=False)

# Salvar números de processos balanceados em CSV
numeros_balanceados = df_decisoes_balanceado['numero_processo'].unique()
with open('numeros_processos.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['numeroProcesso'])
    for num in numeros_balanceados:
        writer.writerow([num])

# Salvar dados completos em JSON
with open('processos_com_decisoes.json', 'w', encoding='utf-8') as f:
    json.dump(decisoes_por_processo, f, indent=2, ensure_ascii=False)

print("Arquivos exportados:")
print("  - processos_completo.csv")
print("  - decisoes_resumo.csv (546 registros balanceados)")
print(f"  - numeros_processos.csv ({len(numeros_balanceados)} processos únicos)")
print("  - processos_com_decisoes.json")

Arquivos exportados:
  - processos_completo.csv
  - decisoes_resumo.csv (546 registros balanceados)
  - numeros_processos.csv (628 processos únicos)
  - processos_com_decisoes.json


# Coleta e Processamento de Dados Judiciais

## Visão Geral

Tem como intuito o processo de coleta e processamento de dados do TJCE.

---

## Etapa 1: Coleta de Dados via API Pública

### 1.1 Configuração Inicial

Como a API do CNJ disponiliza dados estruturados, você precisará:

- **Endpoint**: `https://api-publica.datajud.cnj.jus.br/api_publica_tjce/_search`
- **Autenticação**: APIKey fornecida pelo CNJ
- **Biblioteca**: `requests` para requisições HTTP

### 1.2 Estrutura da Requisição

```python
import requests
import json

payload = {
    "size": 10000,  # Máximo de registros por requisição
    "query": {
        "match": {
            "assuntos.codigo": "12487"  # Código do assunto jurídico
        }
    },
    "sort": [{"dataAjuizamento": {"order": "desc"}}]
}
```

**Obs**: A API retorna dados em formato JSON aninhado (como subpastas). Para debug, printe `response.json()` verifique os níveis `hits > hits > _source`.

### 1.3 Dados Importantes a Extrair

- `numeroProcesso`: Número do processo em questão
- `movimentos`: Array com histórico processual

**Direcionamento**: Implemente uma função que extrai esses campos e os organiza em uma lista de dicionários, para que possamos processá-la depois.

---

## Etapa 2: Processamento com Pandas

### 2.1 Transformação em DataFrame

Converta a lista de processos em um DataFrame do pandas para facilitar análise e filtros:

```python
import pandas as pd

df = pd.DataFrame(processos, columns=['numero_processo', 'movimentos'])
```

### 2.2 Análise de Decisões Judiciais

Para extração de decisão de cada processo

1. Iterar sobre o array `movimentos` de cada processo
2. Identificar movimentos que contenham termos como "Procedência" ou "Improcedência"
3. Criar um novo DataFrame relacionando processos e decisões

**Obs**: Alguns processos têm decisões compostas (ex: "Improcedência do pedido e improcedência do pedido contraposto")

### 2.3 Seleção Estratificada

Para balancear o dataset

```python
# Exemplo de seleção em blocos
procedencias = []
tamanho_bloco = 20
pulo = 60
posicao = 0

while len(procedencias) < target_size:
    procedencias.extend(range(posicao, posicao + tamanho_bloco))
    posicao += tamanho_bloco + pulo
```

### 2.4 Exportação

Salve os resultados em múltiplos formatos:
- CSV com números de processos (entrada para web scraping)
- CSV com decisões mapeadas
- JSON com dados completos

---

## Etapa 3: Web Scraping com Playwright

### 3.1 Por que Web Scraping?

A API não fornece informações sobre juízes e partes do processo. Essas informações estão disponíveis apenas no site do e-SAJ do TJCE.

### 3.2 Configuração do Ambiente

```bash
pip install playwright pandas
playwright install chromium
```

### 3.3 Navegação Automatizada

**Estrutura básica**:

```python
from playwright.async_api import async_playwright
import asyncio

async def buscar_dados_processo(page, numero_processo):
    # 1. Navegar para o site do TJCE
    await page.goto("URL_DO_ESAJ")
    
    # 2. Selecionar tipo de busca
    # 3. Preencher número do processo
    # 4. Clicar em Consultar
    # 5. Aguardar carregamento
    # 6. Extrair informações
```

### 3.4 Extração de Dados

**Desafio - Nome do Juiz**:
- Localizar usando seletores CSS: `#juizPrimeiraDivTable span`
- **Alternativa**: Usar regex no HTML quando não há elemento estruturado
- **Problema comum**: Capturar "Juiz de Direito" ao invés do nome
- **Solução**: Implementar validação de nomes capturados

**Desafio - Nome do Requerente**:
- Buscar em tabela de partes: `#tablePartesPrincipais`
- **Atenção**: O termo pode variar ("Requerente", "Autor", "Massa Falida")
- Implementar busca sequencial com fallback

### 3.5 Sistema de Cache

**Crucial para projetos com centenas de processos**:

```python
# Salvar progresso periodicamente
if idx % 50 == 0:
    salvar_cache(resultados)
```

**Benefícios**:
- Retomar coleta após interrupções
- Evitar reprocessamento
- Adicionar novos processos incrementalmente

### 3.6 Tratamento de Erros

Categorize os resultados:
- `sucesso`: Ambos os dados coletados
- `dados_incompletos`: Apenas um campo encontrado
- `nao_encontrado`: Processo inexistente no sistema
- `erro`: Falha na extração

**Importante**: Filtrar processos não encontrados antes de salvar resultados finais.

---

## Etapa 4: Inferência de Sexo com Base de Dados

### 4.1 Fonte de Dados

Utilize bases públicas de nomes brasileiros (ex: IBGE, bases do GitHub). O formato comum é CSV comprimido (`.csv.gz`).

### 4.2 Leitura de Dados Comprimidos

```python
df_nomes = pd.read_csv(
    'nomes.csv.gz',
    compression='gzip',
    encoding='utf-8',
    on_bad_lines='skip'
)
```

**Tarefa**: Identifique as colunas que contêm o nome e a classificação de sexo.

### 4.3 Estratégia de Inferência

1. **Extração do primeiro nome**:
   ```python
   primeiro_nome = nome_completo.strip().split()[0]
   ```

2. **Normalização**: Converter para maiúsculas para matching case-insensitive

3. **Busca no banco**:
   - Se encontrar uma entrada: retornar sexo
   - Se encontrar múltiplas: usar a classificação mais frequente (`.mode()`)
   - Se não encontrar: retornar "Indefinido"

### 4.4 Aplicação em Lote

Use `apply()` do pandas para processar todos os registros:

```python
df['sexo_juiz'] = df['primeiro_nome_juiz'].apply(
    lambda x: buscar_sexo(x, df_nomes, 'coluna_nome', 'coluna_sexo')
)
```

### 4.5 Validação dos Resultados

Após inferência, verifique:
- Distribuição de sexos (`.value_counts()`)
- Taxa de nomes indefinidos
- Amostras para validação manual

---

## Etapa 5: Integração e Resultados Finais

### 5.1 Estrutura Final do Dataset

```csv
id,numero_processo,juiz,sexo_juiz,requerente,sexo_requerente,sentenca_favoravel,status
1,02187331420258060001,Alda Maria Holanda Leite,F,Heverton Araujo Sena de Assis,M,True,sucesso
```

### 5.2 Fluxo Completo de Arquivos

```
API DataJud → dados_completos.json
           ↓
      Pandas Processing
           ↓
      ├─→ processos_completo.csv (todos os dados)
      ├─→ decisoes_resumo.csv (decisões mapeadas)
      └─→ numeros_processos.csv (input para scraping)
                ↓
          Web Scraping
                ↓
      ├─→ cache_processos.json (progresso)
      └─→ dados_processos_tjce.csv (juiz + requerente)
                ↓
         Inferência de Sexo
                ↓
      dados_processos_com_sexo.csv (dataset final)
```

---

## Boas Práticas e Dicas

### Performance
- Use `headless=True` no Playwright para produção
- Implemente delays (`asyncio.sleep()`) entre requisições
- Processe em lotes para economizar memória

### Qualidade dos Dados
- Valide regex antes de aplicar em produção
- Teste com amostras pequenas primeiro
- Mantenha logs de processos problemáticos

### Reprodutibilidade
- Documente versões de bibliotecas (`requirements.txt`)
- Salve dados intermediários
- Use seeds para amostragens aleatórias

### Ética e Legalidade
- Respeite robots.txt e termos de uso
- Não sobrecarregue servidores públicos
- Anonimize dados sensíveis quando necessário

---

## Exercícios Propostos

1. **API**: Modifique a query para buscar processos de múltiplos assuntos
2. **Regex**: Crie padrões para extrair outras informações (comarca, advogados)
3. **Análise**: Calcule estatísticas sobre tempo de tramitação
4. **Visualização**: Crie gráficos mostrando distribuição de sexo vs. decisão
5. **Machine Learning**: Use o dataset para treinar classificadores de decisão

---

## Estrutura de Arquivos do Projeto

```
projeto/
├── coleta_api.py           # Script de coleta via API
├── processar_dados.py      # Processamento com pandas
├── scraper_tjce.py         # Web scraping automatizado
├── inferir_sexo.py         # Inferência demográfica
├── requirements.txt        # Dependências do projeto
├── dados/
│   ├── input/             # Dados brutos
│   └── output/            # Resultados processados
└── cache/                 # Arquivos temporários
```

---

## Referências Técnicas

- **Playwright**: Documentação oficial em https://playwright.dev/python/
- **Pandas**: Guia de processamento de dados
- **API DataJud**: Documentação do CNJ sobre acesso programático
- **Regex em Python**: Padrões para extração de texto

---

**Nota Final**: Este tutorial fornece a estrutura conceitual e técnica necessária. A implementação específica requer adaptação às peculiaridades de cada tribunal e ajuste fino de parâmetros baseado em experimentação.
